# Import

In [1]:
import pandas as pd
import plotly.graph_objs as go
import string
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
#import cufflinks as cf
from plotly.offline import iplot
from tqdm import tqdm

from collections import Counter
from typing import Dict, Text
from ast import literal_eval
from datetime import datetime
import plotly.express as px

#cf.go_offline()
#cf.set_config_file(offline = False, world_readable = True)

import plotly.io as pio
pio.renderers.default='notebook'

import warnings
warnings.simplefilter('ignore')

In [2]:
from numpy.linalg import svd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

In [3]:
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitItemKNNWrapperModel
from rectools.metrics import (
    Precision,
    Accuracy,
    NDCG,
    IntraListDiversity,
    Serendipity,
    calc_metrics,
)

In [4]:
from implicit.nearest_neighbours import TFIDFRecommender, CosineRecommender

In [5]:
import os
us = os.getcwd()
us

'/Users/liliyaivannikova/Documents/project/git/RecSys_Films'

In [6]:
if 'liliyaivannikova' in us:
    PATH = r'/Users/liliyaivannikova/Documents/project/ml-latest/'
    movies = pd.read_csv(PATH + r'movies.csv')
    rating = pd.read_csv(PATH + r'ratings.csv')
elif 'Владислав' in us:
    movies = pd.read_csv(r'dataset/movies.csv')
    rating = pd.read_csv(r'dataset/ratings.csv')

In [7]:
print(f'''
movies: {movies.shape}
rating: {rating.shape}
''')


movies: (86537, 3)
rating: (33832162, 4)



In [8]:
movies.columns = [col.upper() for col in movies.columns]
movies.head()

,MOVIEID,TITLE,GENRES
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
movies['REALEASE'] = movies['TITLE'].str.extract("\((\d{4})\)", expand=True)
movies['REALEASE'] = pd.to_datetime(movies['REALEASE'], format='%Y')
movies['REALEASE'] = movies['REALEASE'].dt.year
movies['TITLE'] = movies['TITLE'].str[:-7]

In [10]:
movies.head()

,MOVIEID,TITLE,GENRES,REALEASE
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,2,Jumanji,Adventure|Children|Fantasy,1995.0
2,3,Grumpier Old Men,Comedy|Romance,1995.0
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995.0
4,5,Father of the Bride Part II,Comedy,1995.0


In [11]:
dfx = movies['GENRES'].str.get_dummies(sep='|')
for col in dfx.columns:
    dfx[col] = dfx[col].astype('int8')


In [12]:
movies = pd.concat([movies, dfx], axis=1) #.drop(columns=['GENRES'])
movies['TITLE'] = movies['TITLE'].astype('category')
movies['REALEASE'] = movies['REALEASE'].astype('float16')

In [13]:
movies['GENRES'] = movies['GENRES'].str.replace('|', ',')

In [14]:
movies.head()

,MOVIEID,TITLE,GENRES,REALEASE,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,"Adventure,Animation,Children,Comedy,Fantasy",1995.0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,"Adventure,Children,Fantasy",1995.0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,"Comedy,Romance",1995.0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,"Comedy,Drama,Romance",1995.0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II,Comedy,1995.0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [15]:
rating.columns = [col.upper() for col in rating.columns]
rating.head()

,USERID,MOVIEID,RATING,TIMESTAMP
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


# Обработка полей

In [16]:
rating['TIMESTAMP'] = pd.to_datetime(rating['TIMESTAMP'], unit='s')

In [17]:
rating.head()

,USERID,MOVIEID,RATING,TIMESTAMP
0,1,1,4.0,2008-11-03 17:52:19
1,1,110,4.0,2008-11-05 06:04:46
2,1,158,4.0,2008-11-03 17:31:43
3,1,260,4.5,2008-11-03 18:00:04
4,1,356,5.0,2008-11-03 17:58:39


<font color='red'> **Убрал записи  пустыми значениями ~3298, это около 0.01%, фигня короче, можно сносить.** </font>

In [18]:
rating.shape

(33832162, 4)

In [19]:
rating = rating[~((rating.TIMESTAMP.isna())|(rating.USERID.isna())|(rating.RATING.isna()))]

In [20]:
rating['USERID'].astype(str).value_counts().reset_index()[:5] 

,index,USERID
0,189614,33332
1,48766,9554
2,207216,9178
3,175998,9016
4,76618,8919


In [21]:
# при eda нашла странного юзера, уберем его при построении
print(rating.shape)
rating.query('USERID != 189614', inplace = True)
print(rating.shape)

(33832162, 4)
(33798830, 4)


# Collaborative Filtering

In [28]:
def train_test(df, n_days):
    df['weight'] = np.where(df['RATING'] >= 4, 1, 0)
    train_df = df.loc[df['datetime'] < df['datetime'].max() - pd.DateOffset(days=n_days),
                     ['item_id', 'user_id', 'weight', 'datetime']].copy()
    test_df = df.loc[df['datetime'] >= df['datetime'].max() - pd.DateOffset(days=n_days),
                    ['item_id', 'user_id', 'weight', 'datetime']].copy()

    return train_df, test_df

In [23]:
class KNN_ML():

    def __init__(self, model = None, df = None):
        if model is None:
            self.model = ImplicitItemKNNWrapperModel(TFIDFRecommender(K=10))
        else:
            self.model = model
        
        self.df = df
        
    def fit(self, df_train):
        """
            Обучение модели knn
        """
        self.model.fit(df_train)

    def recommend(self, df_train, k):
        """
            Расчет рекомендаций
        """
        prediction = self.model.recommend(users = self.df['user_id'].unique(), 
                                          dataset=df_train,
                                          k = k,
                                          filter_viewed = True
                                        )
        return prediction

In [25]:
rating.rename({'USERID':'user_id',
            'MOVIEID':'item_id',
            #'RATING':'weight',
            'TIMESTAMP':'datetime'}, axis=1, inplace=True)

In [30]:
train_df, test_df = train_test(rating, 180)

In [31]:
train = Dataset.construct(train_df)

In [32]:
test = Dataset.construct(test_df)

In [33]:
model = ImplicitItemKNNWrapperModel(TFIDFRecommender(K=10))

knn_model = KNN_ML(model, train_df)

In [34]:
knn_model.fit(train)

In [35]:
recos = knn_model.recommend(train, k=10)

In [37]:
precision = Precision(k=10)

In [38]:
metrics = {
    "precision": precision
}

In [40]:
calc_metrics(
    metrics,
    reco=recos,
    interactions=test_df,
    prev_interactions=train_df,
    catalog=train_df[Columns.Item].unique()
)

{'precision': 0.0005412252418240442}

In [63]:
del df_test, df_train